In [1]:
import json
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register
from gymnasium.wrappers import FrameStack
from fury import *

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:


def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()


# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11891.96688184432


In [5]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
    
    
cast_metrics = wowsims.getSpellMetrics()

for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 2, 9, 77]
Death Wish: [2]
Deep Wounds: [133]
Whirlwind: [16]
Recklessness: [1]
Bloodthirst: [33]
Battle Stance: [9]
Berserker Stance: [9]
Bloodrage: [4]
Bloodlust: [1]
Whirlwind OH: [16]
Heroic Strike: [68]
Rend: [4]
Execute: [8]
Slam: [19]
Engi Gloves: [3]
Shattering Throw: [1]
Overpower: [5]


# PPO Agent

## Load Trained PPO Model

In [6]:
config = PPOConfig()\
            .rollouts(num_rollout_workers=1)


register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")
# env_creator = lambda config: FrameStack(env, num_stack=5)
env_creator = lambda config: FurySimEnv(...)
register_env("FurySimEnv", env_creator=env_creator)
env = FrameStack(env, num_stack=5)

algorithm = config.build(env="FurySimEnv")
env.spec.max_episode_steps = 10000
algorithm.restore("C:\\Users\\sjsct\\Downloads\\wowsims-warrior-drl-main (1)\\wowsims-warrior-drl-main\\models\\bloodsurge-reward-0.5\\PPO\\PPO_FurySimEnv_5d73b_00000_0_2023-05-05_17-04-03\\checkpoint_002849")

2023-05-09 10:00:36,713	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-09 10:00:42,653	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory C:\Users\sjsct\Downloads\wowsims-warrior-drl-main (1)\wowsims-warrior-drl-main\models\bloodsurge-reward-0.5\PPO\PPO_FurySimEnv_5d73b_00000_0_2023-05-05_17-04-03\checkpoint_002849\.! Trying to extract checkpoint info from other files found in that dir.
2023-05-09 10:00:42,657	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-05-09 10:00:42,674	INFO trainable.py:913 -- Restored on 127.0.0.1 from checkpoint: C:\Users\sjsct\Downloads\wowsims-warrior-drl-main (1)\wowsims-warrior-drl-main\models\bloodsurge-reward-0.5\PPO\PPO_FurySimEnv_5d73b_00000_0_2023-05-05_17-04-03\checkpoint_002849
2023-05-09 10:00:42,675	INFO trainable.py:922 -- Current state after restoring: {'_iteration': 2849, '_timesteps_total': None, '_time_total

## Run PPO Model

In [7]:
import numpy as np

for i in range(100):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    state_out = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

(RolloutWorker pid=60456) 2023-05-09 10:00:42,600	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x120 and 24x256)

In [ ]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

### Debug Log

In [ ]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
    11: "Idle"
}

In [ ]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

In [ ]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [ ]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(1):
    reset()
    while not wowsims.step():
        print(env.reset())
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Bloodthirst)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

In [ ]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")